In [70]:
from scipy.stats import nbinom, binom, poisson
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os
from scipy.optimize import minimize, Bounds, LinearConstraint, NonlinearConstraint
import inspect

In [71]:
path = r'C:/Users/x'
filename = 'Pivot_Trans_Privat_ohneApple_Langzeitkunde.csv'

df_trans_werte = pd.read_csv(os.path.join(path, filename))

In [72]:
path = r'C:/Users/x'
filename = 'Pivot_LY_Privat_ohneApple_Langzeitkunde.csv'

df_LY = pd.read_csv(os.path.join(path, filename))

In [73]:
path = r'C:/Users/x'
filename = 'Pivot_HY_Privat_ohneApple_Langzeitkunde.csv'

df_HY = pd.read_csv(os.path.join(path, filename))

In [74]:
path = r'C:/Users/x'
filename = 'Pivot_NO_Privat_ohneApple_Langzeitkunde.csv'

df_NO_werte = pd.read_csv(os.path.join(path, filename))

In [75]:
path = r'C:/Users/x'
filename = 'Pivot_LO_Privat_ohneApple_Langzeitkunde.csv'

df_LO = pd.read_csv(os.path.join(path, filename))

In [76]:
## Alle DataFrame Werte von Float zu int
df_trans_werte = df_trans_werte.astype(int)
df_LY = df_LY.astype(int)
df_HY = df_HY.astype(int)
df_NO_werte = df_NO_werte.astype(int)
df_LO = df_LO.astype(int)

# Transwerte-1 definiert für spätere Berecnungen in der Copula
#df_trans_werte_minusEins = df_trans_werte.loc[:, df_NO_werte.columns != 'iid'] -1
#df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

### Sicherstellen dass alle df dieselben Kunden verwenden

In [82]:
# Entfernen der Zeilen aus df_HY, die nicht in df_trans_werte und df_LO enthalten sind
df_HY = df_HY[df_HY['iid'].isin(df_trans_werte['iid'])]
df_HY = df_HY[df_HY['iid'].isin(df_LO['iid'])]

# Entfernen der Zeilen aus df_trans_werte, die nicht in df_HY, df_NO_werte und df_LY enthalten sind
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_HY['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_NO_werte['iid'])]
df_trans_werte = df_trans_werte[df_trans_werte['iid'].isin(df_LY['iid'])]

# Entfernen der Zeilen aus df_LO, die nicht in df_HY enthalten sind
df_LO = df_LO[df_LO['iid'].isin(df_HY['iid'])]

# Entfernen der Zeilen aus df_NO_werte, die nicht in df_trans_werte enthalten sind
df_NO_werte = df_NO_werte[df_NO_werte['iid'].isin(df_trans_werte['iid'])]

# Entfernen der Zeilen aus df_LY, die nicht in df_trans_werte enthalten sind
df_LY = df_LY[df_LY['iid'].isin(df_trans_werte['iid'])]

### Alle DataFrames Indizes resetten, damit man sie gemeinsam identifiziere kann
df_HY = df_HY.reset_index(drop=True)
df_trans_werte = df_trans_werte.reset_index(drop=True)
df_LO = df_LO.reset_index(drop=True)
df_NO_werte = df_NO_werte.reset_index(drop=True)
df_LY = df_LY.reset_index(drop=True)

# NO-1 definiert für spätere Berechnung inder Copula
df_NO_minusEins = df_NO_werte.copy()
df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] = df_NO_minusEins.loc[:, df_NO_minusEins.columns != 'iid'] -1
df_NO_minusEins = df_NO_minusEins.astype(int)
# transwerte -1 definiert für spätere Berechnung inder Copula
df_trans_werte_minusEins = df_trans_werte.copy()
df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] = df_trans_werte_minusEins.loc[:, df_trans_werte_minusEins.columns != 'iid'] -1
df_trans_werte_minusEins = df_trans_werte_minusEins.astype(int)

#### Tests

## Ausreißer bei den Kunden löschen

In [ ]:
# Ausreißer wurde vorher ermittelt, weisen außergewöhnlich hohe Ergebisse bei den Öffnugen oder Käufen auf

In [83]:
kunden_pos = [384, 446, 502, 599, 421, 285, 407, 499, 348, 351, 364, 449, 528, 414]

In [84]:
# Zeilen löschen
df_NO_minusEins = df_NO_minusEins.drop(kunden_pos).reset_index(drop = True)
df_HY = df_HY.drop(kunden_pos).reset_index(drop = True)
df_trans_werte = df_trans_werte.drop(kunden_pos).reset_index(drop = True)
df_LO = df_LO.drop(kunden_pos).reset_index(drop = True)
df_NO_werte = df_NO_werte.drop(kunden_pos).reset_index(drop = True)
df_LY = df_LY.drop(kunden_pos).reset_index(drop = True)
df_trans_werte_minusEins = df_trans_werte_minusEins.drop(kunden_pos).reset_index(drop = True)

## Spezifikation des Modells

In [85]:
anzahl_kunden = len(df_trans_werte["iid"])
anzahl_kunden

633

In [86]:
anzahl_monate = len(df_trans_werte.columns[df_trans_werte.columns != 'iid'])
anzahl_monate

15

In [87]:
anzahl_states = 2

##### Definition der Transitionmatrix

In [88]:
# Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat):
    
    state_t_minusEins = state_t_minusEins -1
    state = state -1
    str_monat = str(monat-1)
    
    # Definition der X_t_minusEins
    # verwendete Parameter
    EM_monat_minusEins = df_HY[str_monat]
    Y_monat_minusEins = df_trans_werte[str_monat]
    O_monat_minusEins = df_NO_werte[str_monat]
    # Definition
    X_t_minusEins = [O_monat_minusEins.apply(lambda x: int(x > 0)),\
                     Y_monat_minusEins.apply(lambda x: int(x > 0)),\
                     EM_monat_minusEins, np.square(EM_monat_minusEins)]
    
    X_t_minusEins = np.array(X_t_minusEins)
    array_v = alpha[anzahl_states*state_t_minusEins + state] +\
                    gamma[anzahl_states*state_t_minusEins + state] * X_t_minusEins[0] +\
                    gamma[(np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[1] +\
                    gamma[(2* np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[2] +\
                    gamma[(3* np.square(anzahl_states)) + anzahl_states*state_t_minusEins + state] * X_t_minusEins[3]

    return array_v

# Definition der Transitionmatrix
def probability_Q_state_monat(alpha, gamma):
    
    ### Erstellung leerer anzahl_states x anzahl_states arrays für jeden Kunden und jeden Monat
    Q = np.zeros((anzahl_kunden, anzahl_monate - 1, anzahl_states, anzahl_states))
                
    for monat in range(2, anzahl_monate + 1):
        
        for state_t_minusEins in range(1, anzahl_states +1):
            
            # Erste Spalte besteht aus Nullen, da eine Trasition zurück in state 1 nicht möglich ist
            for state in range(2, anzahl_states +1):
                
                q_array_iid_monat_state = Q_v_state_t_minusEins_state_t_monat(alpha, gamma, state_t_minusEins, state, monat)

                # Speicherung der Werte in den leeren arrays
                for iid in range(0, anzahl_kunden):
                    
                    #Neue Definition ohne die Verwedung von Namen
                    Q[iid, monat-2, state_t_minusEins-1, state-1] = q_array_iid_monat_state[iid]
                    
    # Wenn einmal in state 2, kann nicht mehr zurück in state 1
    Q[:,:,:,0] = 0.0
    
    denominator = np.sum(np.exp(Q[..., 1:]), axis=3, keepdims=True)
    Q = np.exp(Q)/(1+denominator)
    
    return Q

#### Tests

##### Definition der Verteilungsfunktion F_1 von CEOM aals anzahl_kunden x 1 Array für jeden state und Monat

In [93]:
# Definition der Erfolgswahrscheinlichkeit p_t als 1 x anzhal_kunden array 
def lambda_O_state_monat(alpha_o, beta_o, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # EM_iidt:Anzahl dversendeter Mails in Monat t an Kunde mit iid
    EM_monat = df_HY[str_monat]
    LO_monat = df_LO[str_monat]
    
    # Erhalten der Werte für die Verarbeitung in der Formel
    array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))

    return array_lambda

# Array der Verteilungsfunktion der Öffungen O für Monat und state
def F_O_state_monat(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_werte[str_monat]
    
    lambda_O_state_monat_berechnung = lambda_O_state_monat(alpha_o, beta_o, state, monat)
    
    array_F_O_berechnung = poisson.cdf(o_monat, lambda_O_state_monat_berechnung)
         
    return array_F_O_berechnung

# Array der Verteilungsfunktion der Öffungen O -1 für Monat und state
def F_O_state_monat_minusEins(alpha_o, beta_o, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    
    # o_iidt: Anzahl geöffneter Mails in Monat t von allen Kunden
    o_monat = df_NO_minusEins[str_monat]
    
    lambda_O_state_monat_berechnung = lambda_O_state_monat(alpha_o, beta_o, state, monat)
    
    array_F_O_berechnung = poisson.cdf(o_monat, lambda_O_state_monat_berechnung)

    return array_F_O_berechnung

#### Tests

##### Definition der Verteilungsfunktio F_2 von CPM als anzahl_kunden x 1 Array für jeden state und Monat

In [94]:
# Definition der lambdaY als anzahl_iids x 1 array für monat und state
def p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    state = state -1
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    EM_monat = df_HY[str_monat]
    LY_monat = df_LY[str_monat]
    
    array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat) +\
                      + delta_1[state] * np.log(LY_monat))
    array_p = array_exp / (1+ array_exp)
    
    return array_p

# Array der Verteilungsfunktion der Käufe Y für Monat und state
def F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # y_monat ist das array mit der Anzahl an Käufen in monat str_monat
    y_monat = df_trans_werte[str_monat]
    LY_monat = df_LY[str_monat]
    # p ist das 1 x anzahl_iids array mit p für jeden kunden mit gegegebenem monat und state
    array_p = p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
    # cdf berechnen
    cdf = nbinom.cdf(y_monat, 0.779, array_p)
    
    return cdf

# Array der Verteilungsfunktion der Käufe Y-1 für Monat und state
def F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat):
    
    # Definitio von str_monat um auf den spaltennamen zugreifen zu können
    str_monat = str(monat)
    # y_monat ist das array mit der Anzahl an Käufen in monat str_monat
    y_monat = df_trans_werte[str_monat] -1
    LY_monat = df_LY[str_monat]
    # p ist das 1 x anzahl_iids array mit p für jeden kunden mit gegegebenem monat und state
    array_p = p_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
    # cdf berechnen
    cdf = nbinom.cdf(y_monat, 0.779, array_p)
    
    return cdf

#### Tests

##### Definition der Frank-copula

In [95]:
# Frank Copula, theta fehlt
def Frank_copula(u_1, u_2, theta):
    numerator = (np.exp(-theta[0]*u_1) - 1) * (np.exp(-theta[0]*u_2) - 1)
    denominator = np.exp(-theta[0]) - 1
    fraction = numerator / denominator
    return (-1 / theta[0]) * np.log(1 + fraction)

##### Definition der Likelihoodfunktion

In [96]:
# alpha_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2 sind state dependent
# beta_o, r sind state independent
def likelihood_function(params):
    
    alpha_size = anzahl_states**2
    gamma_size = anzahl_states**2 * 4

    alpha = params[:alpha_size]
    gamma = params[alpha_size:alpha_size+gamma_size]
    alpha_o = params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
    beta_o = params[alpha_size+gamma_size+anzahl_states:alpha_size+gamma_size+anzahl_states+1]
    delta_0 = params[alpha_size+gamma_size+anzahl_states+1:alpha_size+gamma_size+anzahl_states*2+1]
    delta_1 = params[alpha_size+gamma_size+anzahl_states*2+1:alpha_size+gamma_size+anzahl_states*3+1]
    alpha_p = params[alpha_size+gamma_size+anzahl_states*3+1:alpha_size+gamma_size+anzahl_states*4+1]
    beta_p_1 = params[alpha_size+gamma_size+anzahl_states*4+1:alpha_size+gamma_size+anzahl_states*5+1]
    beta_p_2 = params[alpha_size+gamma_size+anzahl_states*5+1:alpha_size+gamma_size+anzahl_states*6+1]
    theta = params[alpha_size+gamma_size+anzahl_states*6+1:]


    #alpha, gamma, alpha_o, beta_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, r, theta = params
    ### Erstellung einer Liste H die alle H_it enthalten
    H = np.zeros((anzahl_kunden, anzahl_monate, anzahl_states))
                
                
    for monat in range(1,anzahl_monate+1):
        for state in range(1, anzahl_states +1):
            
            # Berechnung von arrays für die Verteilungsfunktionen
            F_O_state_monat_berechnung = F_O_state_monat(alpha_o, beta_o, state, monat)
            
            F_O_state_monat_berechnung_minusEins = F_O_state_monat_minusEins(alpha_o, beta_o, state, monat)
            
            F_Y_state_monat_berechnung = F_Y_state_monat(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
            
            F_Y_state_monat_berechnung_minusEins =\
                    F_Y_state_monat_minusEins(delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, state, monat)
            
            # Berechnung der gemeinsamen Verteilungen
            H_state_monat_berechnung =\
                            Frank_copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung, theta) -\
                            Frank_copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung, theta) -\
                            Frank_copula(F_O_state_monat_berechnung, F_Y_state_monat_berechnung_minusEins, theta) +\
                            Frank_copula(F_O_state_monat_berechnung_minusEins, F_Y_state_monat_berechnung_minusEins, theta)
            
            # Speicherung der Werte in den leeren arrays
            H[:, monat-1, state-1] = H_state_monat_berechnung

    # Startverteilung ist (1, 0,..., 0)
    startverteilung = np.zeros((1, anzahl_states))
    startverteilung[0, 0] = 1
    
    # Transitionmatrix Q, liste von 3x3 Arrays, wobei für Monate 2, ..., 15 die Q_it angegeben werden und die Q_it fü die
    # nächsten Kunden dahinter angehängt werden
    Q = probability_Q_state_monat(alpha, gamma)
    
    eins_vektor = np.ones((anzahl_states, 1))
    
    likelihood = np.array([])
    
    for iid in range(anzahl_kunden):
        result = startverteilung
        result = np.matmul(result, np.diag(H[iid, 0]))
        H_iid = np.eye(anzahl_states)
        for monat in range(2, anzahl_monate+1):
                H_iid_monat = np.matmul(np.diag(H[iid, monat-1]), Q[iid, monat-2])
                H_iid = np.matmul(H_iid, H_iid_monat)
        result = np.matmul(result, H_iid)
        result = np.matmul(result, eins_vektor)
        likelihood = np.concatenate((likelihood, result[0]))
        
    return abs(np.sum(np.log(likelihood)))

#### Tests

# Berechnung der Likelihoodfunktion und Schätzung der Parameter

### Dispersion Parameter r für die ZINBD Verteilun wurde vorhergeschätzt

In [98]:
r = [1.29]

### Definition der Startwerte

In [99]:
# Parameter für Transitionmatrix, 2 states
values_quelle_alpha = [0, 0, 0, 0]
alpha = np.array(values_quelle_alpha)
gamma = np.zeros((anzahl_states, anzahl_states, 4)).flatten()

# Parameter für CEOM
# values_quelle_alpha_o wurde aus einer vorherigen Optimierung übernommen
values_quelle_alpha_o = [0]
alpha_o = values_quelle_alpha_o * anzahl_states
# values_quelle_beta_o wurdevorher nicht geschätzt
values_quelle_beta_o = [0]
beta_o = values_quelle_beta_o

# Parameter für CPM
# values_quelle_alpha_o wurde aus einer vorherigen Optimierung übernommen
values_quelle_delta_0 = [np.log(1.29)]
delta_0 = values_quelle_delta_0 * anzahl_states
# values_quelle_delta_1 wurde nicht aus einer vorherigen Optimierung übernommen
values_quelle_delta_1 = [0]
delta_1 = values_quelle_delta_1 * anzahl_states
# values_quelle_alpha_p wurde aus einer vorherigen Optimierung übernommen
values_quelle_alpha_p = [1]
alpha_p = values_quelle_alpha_p * anzahl_states
# beta_p_1 wurde nicht aus einer vorherigen Optimierung übernommen
values_quelle_beta_p_1 = [0]
beta_p_1 = values_quelle_beta_p_1 * anzahl_states
# beta_p_2 wurde nicht aus einer vorherigen Optimierung übernommen
values_quelle_beta_p_2 = [0]
beta_p_2 = values_quelle_beta_p_2 * anzahl_states

# Parameter für Frank_copula
theta = np.array([0.324])

In [100]:
# Startwerte
x0 = np.concatenate((
    alpha,
    gamma,
    alpha_o,
    beta_o,
    delta_0,
    delta_1,
    alpha_p,
    beta_p_1,
    beta_p_2,
    theta))

### Sonderfall

In [101]:
#x0 = estimated_params_4

### Definition der Bounds und Constraints

In [102]:
#Definiere bounds
bounds =    [(-3, 3)] * len(alpha) +\
            [(-3, 3)] * len(gamma) +\
            [(-3, 3)] * len(alpha_o) +\
            [(-3, 3)] * len(beta_o) +\
            [(-8, 8)] * len(delta_0) +\
            [(-3, 2)] * len(delta_1) +\
            [(-3, 3)] * len(alpha_p) +\
            [(-3, 3)] * len(beta_p_1) +\
            [(-3, 3)] * len(beta_p_2) +\
            [(0.1, None)] * len(theta)

In [103]:
#Für die Definition der constraint relevant
alpha_size = anzahl_states**2
gamma_size = anzahl_states**2 * 4
# alpha_o = params[alpha_size+gamma_size:alpha_size+gamma_size+anzahl_states]
constraint_matrix = np.zeros((2, len(x0)))
# Positionen für alpha_o
alpha_o_1_pos = alpha_size + gamma_size
alpha_o_2_pos = alpha_size + gamma_size + 1
alpha_o_3_pos = alpha_size + gamma_size + 2
# -1 und 1 an den entsprechenden Positionen setzen
constraint_matrix[0][alpha_o_1_pos] = -1
constraint_matrix[0][alpha_o_2_pos] = 1
constraint_matrix[1][alpha_o_2_pos] = -1
constraint_matrix[1][alpha_o_3_pos] = 1

In [104]:
constraint_matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [105]:
linear_constraint = LinearConstraint(constraint_matrix, [0, 0], [np.inf, np.inf])

### Berechnung per minimize Funktion

In [106]:
options = {'maxiter': 200} 
# Rufen Sie minimize() auf und übergeben Sie die zusätzlichen Parameter mit args=
result = minimize(likelihood_function, x0, options=options, bounds = bounds) 
                  #constraints=linear_constraint)
                  #args=(alpha, gamma, alpha_o, beta_o, delta_0, delta_1, alpha_p, beta_p_1, beta_p_2, r, theta), bounds=bounds)

# Gib die geschätzten Parameter aus
print(result.x)

[ 0.         -2.19735202  0.          2.37972066  0.          1.30515936
  0.          2.58464387  0.          0.1361597   0.         -0.24350485
  0.          1.62238721  0.         -3.          0.         -0.29968159
  0.          0.78316849  0.28249055  1.43691363 -1.08553567  0.25464222
  0.25464222  0.7767828   0.13663523 -0.15226754  0.77264837  0.43711191
  0.07582639 -0.17356914 -0.06043485  1.03787197]


In [107]:
# Berechne die Likelihood für die optimalen Parameter
likelihood = likelihood_function(result.x)
print("Likelihood für optimale Parameter:")
print(likelihood)

Likelihood für optimale Parameter:
20927.456287632907


In [108]:
print(result.success)

False


In [151]:
result

      fun: 20927.456287632907
 hess_inv: <34x34 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00000000e+00, -3.81987777e-02,  0.00000000e+00,  4.32919481e-02,
        0.00000000e+00,  1.49884728e-01,  0.00000000e+00, -7.63975554e-03,
        0.00000000e+00,  1.54977897e-01,  0.00000000e+00, -4.29281499e-02,
        0.00000000e+00, -3.82715373e-01,  0.00000000e+00,  1.50241250e+01,
        0.00000000e+00, -9.45874490e-03,  0.00000000e+00,  2.07364791e-01,
        3.74711817e-01, -5.31144909e-01,  8.05448513e-01,  0.00000000e+00,
        0.00000000e+00,  4.03088050e-01,  3.87808541e-01,  2.58296495e-01,
       -7.27595758e-03,  7.34871719e-02,  1.76805770e-01,  4.29281499e-02,
        1.07102096e+00, -2.71029423e-01])
  message: 'STOP: TOTAL NO. of ITERATIONS REACHED LIMIT'
     nfev: 7945
      nit: 200
     njev: 227
   status: 1
  success: False
        x: array([ 0.        , -2.19735202,  0.        ,  2.37972066,  0.        ,
        1.30515936,  0.        ,  2.5846438

# Ergebisse:

In [122]:
# Ausgangssituation
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda / 1 + array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 17812")

array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda / 1 + array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 17812


In [123]:
# Test ob array_lambda = array_lambda / 1 + array_lambda bessere oder schlechtere Ergebnisse liefert 
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 17812")

array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 17812


In [124]:
# Test kein quadratischer Einfluss des EM auf Käufe
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat)")
print("Likelihood für optimale Parameter: 17812")

array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat)
Likelihood für optimale Parameter: 17812


In [125]:
# Test, ob keine "keine constraints" zu besseren Ergebnissen führt
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 17547")

array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 17547


In [126]:
#Ob nach Anpassungen die schönere Schreibweise auch für die ZINBD Definition funktioeiert und dieselben Ergebnisse liefert
print("likelihood mit ZINBD: 14451")

likelihood mit ZINBD: 14451


In [127]:
# Test ob für Standardspezifikation
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 14451")
print("Parameter:")

likelihood mit ZINBD:
array_lambda = np.exp(np.log(0.63*EM_monat) + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 14451
Parameter:


In [128]:
optim_param = [ 0.00000000e+00, -8.42013774e+01,  1.69986125e+01,  0.00000000e+00,
       -1.25200153e+01, -1.53060979e+01,  0.00000000e+00, -4.01802666e+00,
        3.14514828e+01,  0.00000000e+00, -3.40875207e+01, -2.82844507e-01,
        0.00000000e+00, -1.16977610e+01, -4.21223535e+01,  0.00000000e+00,
       -3.23512569e+01,  5.05520432e+00,  0.00000000e+00, -1.36423722e+00,
       -1.19458664e+00,  0.00000000e+00,  1.05242438e+00, -1.51554184e+01,
        0.00000000e+00,  1.02752609e+00,  8.49398978e-01,  0.00000000e+00,
        7.18296331e+01,  4.41698548e-01,  0.00000000e+00,  1.58841432e+01,
        1.58119714e+01,  0.00000000e+00,  1.02333573e+01, -1.84782829e+01,
        0.00000000e+00, -1.06581824e+01, -4.26447692e-01,  0.00000000e+00,
        2.48374764e+01,  2.47507462e+01,  0.00000000e+00, -1.02161982e+00,
        4.57256224e+00,  2.68038255e-02, -9.78697275e-01,  2.34488631e-01,
       -2.46871753e-01,  1.80545838e+01,  2.12346810e+01,  1.49348847e+01,
        0.00000000e+00,  1.72348151e-01, -3.64387979e-01, -5.53773619e-01,
       -2.76890669e+00, -1.66599130e+00,  4.94843823e-01,  8.64751297e-01,
       -6.48676929e-02, -2.92112867e-01, -1.55380568e-01,  5.61001901e-02,
        1.47689589e+00]

In [129]:
# Test ob np.log(0.63)*EM_monat besser als np.log(0.63*EM_monat)läuft und evtl terminiert
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18522")

likelihood mit ZINBD:
array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 18522


In [130]:
optim_param = [ 0.00000000e+00, -8.42324287e+01,  1.69437089e+01,  0.00000000e+00,
       -1.51114259e+01, -1.27787560e+01,  0.00000000e+00, -4.66039409e+00,
        3.23378060e+01,  0.00000000e+00, -3.42059247e+01, -3.03032101e-01,
        0.00000000e+00, -1.16969644e+01, -4.21223535e+01,  0.00000000e+00,
       -3.23512569e+01,  5.10330353e+00,  0.00000000e+00, -1.34546356e+00,
       -1.45060842e+00,  0.00000000e+00,  1.05242438e+00, -1.51554184e+01,
        0.00000000e+00,  6.50267936e-01,  1.11128688e+00,  0.00000000e+00,
        7.18149941e+01,  5.07482709e-01,  0.00000000e+00,  1.65202863e+01,
        1.50351002e+01,  0.00000000e+00,  9.40601150e+00, -1.66324994e+01,
        0.00000000e+00, -1.05955017e+01, -1.72078495e-01,  0.00000000e+00,
        2.62869932e+01,  2.28266666e+01,  0.00000000e+00, -1.69422693e+00,
        7.29303793e+00,  4.87220599e-01,  7.01288616e-01,  1.24299189e+00,
       -1.36993482e+00,  1.80247785e+01,  2.12346810e+01,  1.55971024e+01,
        0.00000000e+00,  1.72348151e-01, -5.49982853e-01, -2.11311398e-01,
       -3.31995657e+00, -1.50224012e+00,  2.04106841e-01,  1.71997448e+00,
       -8.41039175e-02, -2.74031043e-01, -4.31434986e-01,  1.32125605e-02,
        1.07630364e+00]

In [131]:
# Test ob np.log(0.63)*EM_monat besser als np.log(0.63*EM_monat)läuft und evtl terminiert
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18522")
print("result.success: False")

likelihood mit ZINBD:
array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 18522
result.success: False


#### Einzige Interpretierbare Ergebnisse: ZINBD und 2 states

In [132]:
interpretierbare_param = [  0.        , -18.15053611,   0.        , -58.29432985,
         0.        ,  -2.90055678,   0.        ,  14.34443851,
         0.        ,   0.45459921,   0.        , -15.93902142,
         0.        ,  26.45959452,   0.        ,  44.41306213,
         0.        ,  -7.85441467,   0.        , -11.04343658,
         0.73832154,   1.79246772,  -1.25828191,   8.15200577,
        14.28575254,  -0.76026437,  -1.58603474,  -1.21677622,
        -2.79459761,  -0.25805106,   0.73952882,   0.08644445,
        -0.11602983,   0.81965572]

In [133]:
print("likelihood: 17988")
print("options = {'maxiter': 100}") 
print("result = minimize(likelihood_function, x0, options=options)")
print(" success: False")

likelihood: 17988
options = {'maxiter': 100}
result = minimize(likelihood_function, x0, options=options)
 success: False


### Weitere Testsmit 2 states

In [134]:
# Test für 2 states und 1000 Iterationen und bounds
print("likelihood mit ZINBD:")
print("array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))")
print("array_lambda = array_lambda")
print("array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))")
print("Likelihood für optimale Parameter: 18293")
print(" success: True")

likelihood mit ZINBD:
array_lambda = np.exp(np.log(0.63)*EM_monat + alpha_o[state] + beta_o[0] * np.log(LO_monat))
array_lambda = array_lambda
array_exp = np.exp(alpha_p[state] + beta_p_1[state] * EM_monat + beta_p_2[state] * np.square(EM_monat))
Likelihood für optimale Parameter: 18293
 success: True


In [135]:
results_param = [ 0.        ,  3.13291287,  0.        ,  1.60263079,  0.        ,
        0.06988214,  0.        ,  0.79140532,  0.        ,  0.91950281,
        0.        , -1.10907081,  0.        , -3.41801979,  0.        ,
       -3.41917121,  0.        , -4.23594507,  0.        , -3.9000474 ,
        1.10129257, -5.        , -0.79042102,  5.        ,  5.        ,
        4.63925216,  1.95999501, -0.53922688, -1.71813938, -0.86955723,
       -1.45737121,  0.18195139,  1.1       ,  0.90651293]